In [1]:
PATH = "/diskA/jethro/cnn/"

In [2]:
RAW = f'{PATH}/raw'

In [3]:
import os

In [4]:
stories = os.listdir(RAW)

In [5]:
len(stories)

92579

In [6]:
STORY_LENGTH          = 100
NUM_SUMMARIES_TO_SAVE = 1

In [7]:
processed_stories = []
for s in stories:
    tt = open(f'{RAW}/{s}').readlines()
    tt = [t.replace("-LRB- CNN Student News -RRB- --", "")
          .replace("-LRB- CNN -RRB- --", "")
          .replace("-LRB-", "")
          .replace("-RRB-", "")
          .replace("''", '"')
          .replace("``", '"')
          .strip() for t in tt if t != "\n"]
    paragraph = []
    summary = []
    flag = True
    for i, t in enumerate(tt):
        if t == "@highlight":
            flag = False
            summary.append(tt[i+1])
        elif flag:
            paragraph.append(t)
    paragraph = " ".join(" ".join(paragraph).split()[:STORY_LENGTH])
    summary = " ".join(summary[:NUM_SUMMARIES_TO_SAVE])
    if len(paragraph.strip()) > 0 and len(summary.strip()) > 0:
        processed_stories.append({
            "story": paragraph,
            "summary": summary
        })

In [8]:
len(processed_stories)

92465

In [9]:
import pandas as pd

In [10]:
df = pd.DataFrame(processed_stories)

In [11]:
df.head()

,story,summary
0,"Fort Hood , Texas The shooting at Fort Hood wa...",NEW : Pfc. Lance Aviles says he was ordered to...
1,WikiLeaks founder Julian Assange is trying to ...,Julian Assange is seeking to avoid extradition...
2,The news that Scotland has rejected independen...,Hammond : UK allies will breathe sigh of relie...
3,BP plans to continue using a controversial sub...,"EPA says it "" will continue to work over the n..."
4,Barcelona may be licking their wounds after a ...,Barcelona 's Eric Abidal is given the all clea...


In [12]:
df.describe()

,story,summary
count,92465,92465
unique,89256,88318
top,"ATLANTA , Georgia Seventh graders at Ron Clark...",This page includes the show Transcript
freq,4,148


In [21]:
df.iloc(0)[0][0]

'Fort Hood , Texas The shooting at Fort Hood was captured on video by a soldier using his cell phone camera as he hid from the shooter , but he was ordered to erase it , the soldier said Friday . Pfc. Lance Aviles spoke of the video as he testified on the third day of the Article 32 military hearing for Maj. Nidal Hasan , who is accused of killing 13 people and wounding 32 in the November 2009 shooting . Aviles said he was told by a non-commissioned officer , who Aviles said was acting on the orders'

In [22]:
df.to_feather(f'{PATH}/stories.feather')

In [23]:
from sklearn.model_selection import train_test_split

In [24]:
train, test = train_test_split(df, test_size=0.2)
test, valid = train_test_split(test, test_size=0.2)

In [25]:
train = train.reset_index()
train.to_feather(f'{PATH}/train.feather')

In [26]:
test = test.reset_index()
test.to_feather(f'{PATH}/test.feather')

In [27]:
valid = valid.reset_index()
valid.to_feather(f'{PATH}/valid.feather')

In [28]:
train.to_csv(f'{PATH}/train.tsv', sep="\t", columns=["story", "summary"], index=False, header=False)

In [29]:
test.to_csv(f'{PATH}/test.tsv', sep="\t", columns=["story", "summary"], index=False, header=False)

In [30]:
valid.to_csv(f'{PATH}/valid.tsv', sep="\t", columns=["story", "summary"], index=False, header=False)

In [31]:
len(train), len(test), len(valid)

(73972, 14794, 3699)